In [2]:
import math
from pathlib import Path

from PIL import Image, ImageFilter
from tqdm import tqdm

from MaterialEdit.fun_图片编辑.fun_图片扩大粘贴 import fun_图片扩大粘贴
from MaterialEdit.setting import IMAGE_SUFFIX

yy = Image.open("ipynb/IMG/折页阴影.png")


def fun_自动画册贴图(img_path: str, col: int) -> Image.Image:
    """画册添加阴影."""
    im = Image.open(img_path)
    if im.mode.lower() != "rgba":
        im = im.convert("RGBA")

    bg = Image.new("RGBA", im.size, (255, 255, 255, 255))
    bg.paste(im, (0, 0), im)

    yy.thumbnail(im.size, resample=Image.Resampling.LANCZOS)

    small_width = math.floor(im.width / col)

    if col in (3, 2):
        left_list = [0, small_width, small_width + small_width]

    top = 0
    r, g, b, a = yy.split()
    for left in left_list:
        while top < im.height:
            bg.paste(yy, (left, top), a)
            top += yy.height

        top = 0
    return bg


def fun_图片添加阴影(im: Image.Image) -> Image.Image:
    """单个图片添加阴影."""
    mask = Image.new("RGBA", im.size, (160, 160, 160, 255))
    out = math.ceil(im.width * 0.1)
    mask = fun_图片扩大粘贴(
        im=mask,
        width=im.width + out,
        height=im.height + out,
        left="center",
        top="center",
        background_color=(255, 255, 255, 255),
    )
    mask = mask.filter(ImageFilter.GaussianBlur(radius=10))

    left = math.floor(((mask.width - im.width) / 2) - (im.width * 0.003))
    top = math.floor(((mask.height - im.height) / 2) - (im.height * 0.003))
    mask.paste(im, (left, top), im)

    return mask


for in_file in tqdm(
    list(Path(r"F:\小夕素材\10000-20000\10968\预览图").rglob("*")),
    desc="画册",
    ncols=200,
):
    if in_file.is_file() and in_file.suffix.lower() in IMAGE_SUFFIX and "thumb" not in in_file.stem:
        im = fun_自动画册贴图(img_path=in_file.as_posix(), col=2)
        im = fun_图片添加阴影(im=im)

        if in_file.suffix.lower() in [".jpg", ".jpeg"]:
            im = im.convert("RGB")
            im.save(in_file.as_posix(), quitly=80)
        else:
            im.save(in_file.as_posix())

        im.close()


画册: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 494/494 [03:16<00:00,  2.51it/s]
